In [189]:
import os
import pandas as pd
import numpy as np
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
import yfinance as yf

In [190]:
ticker = 'CL=F' # stock ticker symbol: WTI Crude Oil
period = 'max' # time range to look at: all time
interval = '1d' # time interval for data collected: daily

In [191]:
def get_hist(ticker, time, interval):
    tick = yf.Ticker(ticker)
    return tick.history(period=time, interval=interval)

In [192]:
# get historic data
df = get_hist(ticker, period, interval)

In [193]:
# reversing data so top is most recent
df = df.loc[::-1]

In [194]:
date = df.reset_index()['Date']

# add dates as dataframe columns
df['year'] = date.dt.year.values
df['month'] = date.dt.month.values
df['day'] = date.dt.day.values

# gets rid of datetime indexing
df = df.reset_index()
df = df.drop('Date', axis=1)

In [195]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,month,day
0,102.529999,106.239998,101.239998,105.150002,79533600,0.0,0.0,2023,2,1
1,101.160004,103.349998,101.139999,103.129997,66527300,0.0,0.0,2023,1,31
2,101.089996,101.739998,99.010002,100.550003,70691900,0.0,0.0,2023,1,30
3,99.529999,103.489998,99.529999,102.239998,87678100,0.0,0.0,2023,1,27
4,98.239998,99.489998,96.919998,99.220001,68523600,0.0,0.0,2023,1,26


In [196]:
test = df.drop(df.tail(5000).index, inplace = False)
train = df
train_y = train['Open']
train_X = train.drop(columns='Open')
test_y = test['Open']
test_X = test.drop(columns='Open')

In [197]:
# dates = {
#     'year':[],
#     'month':[],
#     'day':[]
# }
# for i in range(5):
#     dates['year'] += [2023]
#     dates['month'] += [1]
#     dates['day'] += [25+i]
# future = pd.DataFrame.from_dict(dates)
# future

In [198]:
# Instantiation
xgb_r = xg.XGBRegressor(objective='reg:squarederror', n_estimators=10, seed=123)

In [199]:
# Fitting the model
xgb_r.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [200]:
# Predict the model
pred = xgb_r.predict(test_X)

In [201]:
# RMSE Computation
rmse = np.sqrt(MSE(test_y, pred))
print("RMSE : % f" %(rmse))

RMSE :  3.677233


In [ ]:
# todo:
# setup a prediction view method
# predict future
# make test set the last year instead of 5000